In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [12]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

In [13]:
!tar xf spark-3.3.1-bin-hadoop3.tgz

In [9]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 32.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=c0fb1b10dfd52ac0269f6df9fce8512cd182040716351d94c292144c5ab777dd
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [15]:
!pip install -q findspark

In [20]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"
os.environ["HADOOP_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [18]:
!unzip /content/TCU.zip

Archive:  /content/TCU.zip
  inflating: TCU/103_1.txt           
  inflating: TCU/114_1.txt           
  inflating: TCU/132_0.txt           
  inflating: TCU/134_1.txt           
  inflating: TCU/141_1.txt           
  inflating: TCU/148_4.txt           
  inflating: TCU/149_1.txt           
  inflating: TCU/150_6.txt           
  inflating: TCU/153_1.txt           
  inflating: TCU/154_0.txt           
  inflating: TCU/157_1.txt           
  inflating: TCU/158_0.txt           
  inflating: TCU/160_0.txt           
  inflating: TCU/164_8.txt           
  inflating: TCU/17_2.txt            
  inflating: TCU/170_1.txt           
  inflating: TCU/172_0.txt           
  inflating: TCU/176_0.txt           
  inflating: TCU/18_0.txt            
  inflating: TCU/180_2.txt           
  inflating: TCU/21_0.txt            
  inflating: TCU/23_1.txt            
  inflating: TCU/24_1.txt            
  inflating: TCU/26_2.txt            
  inflating: TCU/28_0.txt            
  inflating: TCU/30_1.t

In [49]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()


def getListOfReqData(filePath, headerString):
    myList = []
    content = []
    pretext = []
    flag = True
    with open(filePath, encoding='utf-8') as f:
        r = 0
        for line in f.readlines():
            if flag == False:
                if len(line.strip()) == 0:
                    continue
                else:
                    myList.append(line.strip())
                    flag = True
            if len(line.strip()) != 0 and "University" not in line and len(myList) == 0:
                pretext.append(line.strip())
                continue
            if "University" in line and len(pretext) != 0 and not (line.strip().endswith("University") or line.strip().endswith("University Policy")) and len(myList) == 0:
                pretext.append(line.strip())
                continue
            if "University" in line and len(myList) == 0:
                myList.append(line.strip())
                continue

            while r < len(headerString[0]):

                if line.startswith(headerString[0][r]) or line.startswith(headerString[1][r]): 
                    if r == 3:
                        myList.append(line[line.index(":")+1:].strip())
                    elif len(line[line.index(":")+1:].strip()) != 0:
                        myList.append(line[line.index(":")+1:].strip())
                    else:
                        flag = False
                    r += 1
                else: break

            else:
                if r == len(headerString[0]):
                    r = float("inf")
                    continue        
                if len(line.strip()) != 0:
                    content.append(line.strip())

        myList.append(" ".join(content))
        myList.append(" ".join(pretext))
        if len(myList) != 7:
            title = pretext[0]
            myList = ["", title, "", "", "", "".join(pretext[1:]), ""]


    f.close()
    return(myList)

headerString =  [["Policy Title","Policy Subject", "Division", "Policy Number"],["Título de la política", "Asunto", "División", "Número de política"]]

#path for TCU file
path = "/content/TCU"

os.chdir(path)
dataList = []


typel = []
for file in os.listdir():
    data = []
    if file.endswith(".txt"):
        filePath = f"{path}/{file}"
        dataElement = getListOfReqData(filePath, headerString)
        dataElement = tuple(dataElement)

        for i in dataElement:
            if len(i) == 0:
                data.append("null")
            else:
                data.append(i)
                typel.append(type(i))
        data.append(file)
        data = tuple(data)
        dataList.append(data)


DF = spark.createDataFrame(dataList,schema=['University', 'Policy Title', 'Policy Subject', 'Division', 'Policy Number', 'Content', 'pretext','filename'])
DF.cache()

DF.show(50, truncate=100)

DF.repartition(1).write.format('com.databricks.spark.csv').save("/content/dataframe.csv",header = 'true')



+--------------------------------------+-------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------+-------------------------------------------------------------------------+-----------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+---------+
|                            University|                                                                         Policy Title|                                                                         Policy Subject|                                                                 Division|                Policy Number|                                                                                             Content|                                                  

In [47]:
%cd /content/dataaa.csv
%cd /content
%ls
%rm -r data.csv

[Errno 2] No such file or directory: '/content/dataaa.csv'
/content
/content
create_dataframe.py  drive/        spark-3.3.1-bin-hadoop3/     TCU/     xf.zip
data.csv/            sample_data/  spark-3.3.1-bin-hadoop3.tgz  TCU.zip
